17- just to put everyone together

In [149]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import xarray as xr
import json
import pickle
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
%matplotlib widget


Load multiple session

In [150]:
folder_base = Path('Gaelle/Baseline_recording/BlackLinesOK/')
sessions = 4

mfile = open('Gaelle/Baseline_recording/BlackLinesOK/mappings.pkl', 'rb')
mapping = pickle.load(mfile)

dict_Int = {}
dict_Counter = {}
for x in range(1, sessions+1):
    IntensityFile = folder_base / f'session{x}/CTRL_Spikes_spindlesSWRInd_14bis.xlsx' #
    dict_Int["session{0}".format(x)] = pd.read_excel(IntensityFile)
    CounterFile = folder_base / f'session{x}/CounterSWR_16bis.xlsx' #
    dict_Counter["session{0}".format(x)] = pd.read_excel(CounterFile)


In [151]:
units = len(mapping)
B = mapping['session']
# this is only necessary in case the final session analysed are not the same as used for cross-registration
B = B[['session1', 'session2', 'session3', 'session4']]#, 'session5']] 

data = {
    'NegAll': np.zeros(units),
    'NegBefore': np.zeros(units),
    'NegAfter' : np.zeros(units),
    'PosAll' : np.zeros(units),
    'PosBefore' : np.zeros(units),
    'PosAfter' : np.zeros(units),
    'UnAll' : np.zeros(units),
    'UnBefore' : np.zeros(units),
    'UnAfter' : np.zeros(units),
    'Sessions' : np.zeros(units)
}
Activity_Per_Neurons = pd.DataFrame(data, columns=['NegAll', 'NegBefore', 'NegAfter', 'PosAll', 'PosBefore', 'PosAfter', 'UnAll', 'UnBefore', 'UnAfter', 'Sessions'])
counterNegSWR = 0
counterPosSWR = 0
counterUnSWR = 0

for unit in range(units):
    counter = 0
    for i in B:
        counterNegSWR += list(dict_Counter[i][0])[0]
        counterPosSWR += list(dict_Counter[i][0])[1]
        counterUnSWR += list(dict_Counter[i][0])[2]
        if math.isnan (float(B[i][unit])) == False:
            counter = counter + 1
            unit_id = int(float(B[i][unit]))
            Activity_Per_Neurons['Sessions'][unit] = counter
            Unit_id_rank = [x for x in range(len(dict_Int[i]['UnitId'])) if dict_Int[i]['UnitId'][x] == unit_id]
            if Unit_id_rank != [] and Activity_Per_Neurons['Sessions'][unit] > 0:
                Activity_Per_Neurons['NegAll'][unit] += dict_Int[i]['NegAll'][Unit_id_rank]*list(dict_Counter[i][0])[0]
                Activity_Per_Neurons['NegBefore'][unit] += dict_Int[i]['NegBefore'][Unit_id_rank]*list(dict_Counter[i][0])[0]
                Activity_Per_Neurons['NegAfter'][unit] += dict_Int[i]['NegAfter'][Unit_id_rank]*list(dict_Counter[i][0])[0]
                Activity_Per_Neurons['PosAll'][unit] += dict_Int[i]['PosAll'][Unit_id_rank]*list(dict_Counter[i][0])[1]
                Activity_Per_Neurons['PosBefore'][unit] += dict_Int[i]['PosBefore'][Unit_id_rank]*list(dict_Counter[i][0])[1]
                Activity_Per_Neurons['PosAfter'][unit] += dict_Int[i]['PosAfter'][Unit_id_rank]*list(dict_Counter[i][0])[1]
                Activity_Per_Neurons['UnAll'][unit] += dict_Int[i]['UnAll'][Unit_id_rank]*list(dict_Counter[i][0])[1]
                Activity_Per_Neurons['UnBefore'][unit] += dict_Int[i]['UnBefore'][Unit_id_rank]*list(dict_Counter[i][0])[1]
                Activity_Per_Neurons['UnAfter'][unit] += dict_Int[i]['UnAfter'][Unit_id_rank]*list(dict_Counter[i][0])[1]
Activity_Per_Neurons

Activity_Per_Neurons['NegAll'] /= counterNegSWR
Activity_Per_Neurons['NegBefore'] /= counterNegSWR
Activity_Per_Neurons['NegAfter'] /= counterNegSWR
Activity_Per_Neurons['PosAll'] /= counterPosSWR
Activity_Per_Neurons['PosBefore'] /= counterPosSWR
Activity_Per_Neurons['PosAfter'] /= counterPosSWR
Activity_Per_Neurons['UnAll'] /= counterUnSWR
Activity_Per_Neurons['UnBefore'] /= counterUnSWR
Activity_Per_Neurons['UnAfter'] /= counterUnSWR

filenameOut = folder_base / f'CTRL_S_CouplingSPI14_Consolidated.xlsx'
writer = pd.ExcelWriter(filenameOut)
Activity_Per_Neurons.to_excel(writer)
writer.save()